<a href="https://colab.research.google.com/github/dialloibrahimakhaliloulaye/IA/blob/master/chatbotProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programmation d'un simple Chatbot de Scratch en Python (utilisation de NLTK)


![Alt text](https://cdn-images-1.medium.com/max/800/1*pPcVfZ7i-gLMabUol3zezA.gif)

l'histoire des chatbots remonte à 1966 quand un programme informatique appelé ELIZA a été inventé par Weizenbaum. Il a imité le langage d'un psychothérapeute à partir de seulement 200 lignes de code. Vous pouvez toujours en parler ici: [Eliza](http://psych.fullerton.edu/mbirnbaum/psych101/Eliza.htm?utm_source=ubisend.com&utm_medium=blog-link&utm_campaign=ubisend). 

Sur des lignes similaires, créons un chatbot très basique utilisant la bibliothèque NLTK de Python.C'est un bot très simple avec presque aucune compétence cognitive, mais toujours un bon moyen d'entrer en NLP et de se familiariser avec les chatbots.


## NLP
La NLP est un moyen pour les ordinateurs d'analyser, de comprendre et de tirer un sens du langage humain d'une manière intelligente et utile. En utilisant la PNL, les développeurs peuvent organiser et structurer leurs connaissances pour effectuer des tâches telles que la synthèse automatique, la traduction, la reconnaissance d'entités nommées, l'extraction de relations, l'analyse de sentiments, la reconnaissance vocale et la segmentation de sujets.

##Importations des librairies nécessaires

In [47]:
import nltk
import numpy as np
import random
import string # to process standard python strings


## Lire dans le corpus

Pour notre exemple, nous utiliserons la page Wikipedia pour les chatbots comme corpus. Copiez le contenu de la page et placez-le dans un fichier texte nommé ‘racisme.txt’ que nous allons placer dans le drive (content/dirive/My drive/racisme.txt). Cependant, vous pouvez utiliser n'importe quel corpus de votre choix.

In [48]:
f=open('/content/drive/My Drive/racisme.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase
nltk.download('punkt') # first-time use only
nltk.download('wordnet') # first-time use only
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Droit d'accès au drive
on doit d'abord monter le drive ensuite donner les autorisations d'accès à notre fichier 'racisme.txt

In [49]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## La tokenisation
on doit envoyé les tokens

In [50]:
sent_tokens[:2]
['a chatbot (also known as a talkbot, chatterbot, bot, im bot, interactive agent, or artificial conversational entity) is a computer program or an artificial intelligence which conducts a conversation via auditory or textual methods.',
 'such programs are often designed to convincingly simulate how a human would behave as a conversational partner, thereby passing the turing test.']
word_tokens[:2]
['a', 'chatbot', '(', 'also', 'known']

['a', 'chatbot', '(', 'also', 'known']

##Prétraitement
Nous allons maintenant définir une fonction appelée LemTokens qui prendra en entrée les jetons et retournera les jetons normalisés

In [51]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

##Matching des mots clés
Ensuite, nous allons définir une fonction pour un message d'accueil par le bot, c'est-à-dire que si la saisie d'un utilisateur est un message d'accueil, le bot doit retourner une réponse de salutation.ELIZA utilise un simple mot clé correspondant pour les salutations. Nous utiliserons ici le même concept.

In [52]:
GREETING_INPUTS = ("Bonjour", "merci", "salut", "slt", "vous allez bien","ca va","Je vous remercie")
GREETING_RESPONSES = ["assalamou aleykoum", "ca va", "*nods*", "bonjour ici", "hello", "bienvenue chez moi, merci de poser vos questions"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

## Génération d'une réponse

### Bloc de mots
Après la phase initiale de prétraitement, nous devons transformer le texte en un vecteur (ou tableau) significatif de nombres. Le sac de mots est une représentation de texte qui décrit l'occurrence de mots dans un document. Cela implique deux choses:

* Un vocabulaire de mots connus.

* Une mesure de la présence de mots connus.

Pourquoi est-il appelé un «sac» de mots? En effet, toute information sur l'ordre ou la structure des mots dans le document est supprimée et le modèle est uniquement ** concerné par la question de savoir si les mots connus se trouvent dans le document, pas où ils se trouvent dans le document. **

L'intuition derrière le sac de mots est que les documents sont similaires s'ils ont un contenu similaire. En outre, nous pouvons apprendre quelque chose sur la signification du document à partir de son seul contenu.

Par exemple, si notre dictionnaire contient les mots {Learning, is, the, not, great}, et que nous voulons vectoriser le texte "Learning is great", nous aurions le vecteur suivant: (1, 1, 0, 0, 1).

In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer


import cosine

In [54]:
from sklearn.metrics.pairwise import cosine_similarity

Pour générer une réponse de notre bot aux questions d'entrée, le concept de similitude des documents sera utilisé. Nous définissons une réponse de fonction qui recherche dans l'énoncé de l'utilisateur un ou plusieurs mots clés connus et renvoie l'une des réponses possibles. S'il ne trouve pas l'entrée correspondant à l'un des mots clés, il renvoie une réponse: "Je suis désolé! Je ne te comprends pas "

In [55]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"Désolé! mais je ne vous comprends pas, merci de réessayer"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

##Lignes d'alimentation
Enfin, nous alimenterons les lignes que nous voulons que notre bot dise tout en commençant et en terminant une conversation en fonction de l'entrée de l'utilisateur.

In [ ]:
flag=True
print("ROBO: Mon nom est Robo. Je vais essayer de répondre à vos questions dans le Chatbots. Si vous voulez quitez, tapez Bye!")
while(flag==True):
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='merci' or user_response=='Je vous remercie' ):
            flag=False
            print("ROBO: Grand merci à vous..")
        else:
            if(greeting(user_response)!=None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! A plus..")

ROBO: Mon nom est Robo. Je vais essayer de répondre à vos questions dans le Chatbots. Si vous voulez quitez, tapez Bye!


/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


ROBO: etymologie
selon le cnrtl, le mot racisme serait apparu en 19028 alors que le mot raciste daterait de 18929.


selon charles maurras  ; gaston méry (1866-1909), pamphlétaire, journaliste collaborateur a la libre parole — le journal 
antisémite et polémiste d'edouard drumont — est la première personne connue à avoir utilisé le mot « raciste » en
 189411,12,13.


toutefois l'adjectif « raciste »14 et le nom « racisme » ne s'installent dans le vocabulaire général en france qu'à 
partir des années 193015. léon trotsky l'emploie en 1930 dans son histoire de la révolution russe, avec un sens culturel 
pour qualifier le groupe des slaves traditionalistes qui défendent leur culture et leur mode de vie national.
